In [4]:
!pip install  face-recognition 
# python3 faceforensics_download_v4.py -d 'DeepFakeDetection'  data --server EU2 --c c40

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached dlib-19.24.8.tar.gz (3.4 MB)
  Preparing metadata (setup.py) ... done
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
  Created wheel for dlib: filename=dlib-19.24.8-cp311-cp311-linux_x86_64.whl size=4145126 sha256=00cc317b33713e28040fa71c6548d84f536d656d848dcd560de20dd526f5e297
  Stored in directory: /home/alex/.cache/pip/wheels/45/df/3c/d9d5185a237d3c39571ab9a9fac6b984927b826d80dbd1569b
Successfully built dlib


In [7]:
 import os

os.getcwd()

'/home/intern/alex/exception-ffc'

In [1]:
import tensorflow as tf

2025-04-24 18:25:47.287288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745508347.302366 2004477 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745508347.307001 2004477 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745508347.319355 2004477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745508347.319373 2004477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745508347.319375 2004477 computation_placer.cc:177] computation placer alr

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
filtered = gpus[2:5]
print (filtered)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU')]
[PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU')]


In [3]:
tf.config.experimental.set_visible_devices(filtered, 'GPU')  # Используем только 2 GPU

# (опц.) Устанавливаем память на рост по запросу
for gpu in gpus[:2]:
    tf.config.experimental.set_memory_growth(gpu, True)

In [9]:
# DeepFake Detection using XceptionNet fine-tuned on DeepFakeDetection dataset


import os
import cv2
import numpy as np
import face_recognition
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# === Настройки ===
DATASET_PATH = '/home/intern/alex/exception-ffc/dataset'
REAL_DIR = '/home/intern/alex/exception-ffc/data/original_sequences/actors/c40/videos'
FAKE_DIR = '/home/intern/alex/exception-ffc/data/manipulated_sequences/DeepFakeDetection/c40/videos'
IMG_SIZE = 299
BATCH_SIZE = 8
EPOCHS = 5

os.makedirs(f"{DATASET_PATH}/real", exist_ok=True)
os.makedirs(f"{DATASET_PATH}/fake", exist_ok=True)

# === Извлечение лиц из видео ===
def extract_faces_from_video(video_path, save_dir, label_prefix, max_faces=10):
    cap = cv2.VideoCapture(video_path)
    saved = 0
    frame_id = 0
    while saved < max_faces:
        ret, frame = cap.read()
        if not ret:
            break
        face_locations = face_recognition.face_locations(frame)
        if not face_locations:
            continue
        top, right, bottom, left = face_locations[0]
        face = frame[top:bottom, left:right]
        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
        fname = f"{label_prefix}_{frame_id}.jpg"
        cv2.imwrite(os.path.join(save_dir, fname), face)
        saved += 1
        frame_id += 1
    cap.release()

# === Обработка реальных и фейковых видео ===
real_videos = [f for f in os.listdir(REAL_DIR) if f.endswith('.mp4')][:50]  # ограничим по 5 видео
fake_videos = [f for f in os.listdir(FAKE_DIR) if f.endswith('.mp4')][:50]

for fname in tqdm(real_videos, desc='Extracting real'):
    extract_faces_from_video(os.path.join(REAL_DIR, fname), f"{DATASET_PATH}/real", fname[:-4])

for fname in tqdm(fake_videos, desc='Extracting fake'):
    extract_faces_from_video(os.path.join(FAKE_DIR, fname), f"{DATASET_PATH}/fake", fname[:-4])

# === Построение модели ===
base_model = Xception(weights=None, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.load_weights('/home/intern/alex/exception-ffc/xception_weights_tf_dim_ordering_tf_kernels_notop.h5')  # загрузи вручную через панель файлов
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# === Загрузка изображений и генерация выборки ===
def load_images_from_directory(base_dir, label):
    images = []
    labels = []
    for fname in os.listdir(base_dir):
        fpath = os.path.join(base_dir, fname)
        img = cv2.imread(fpath)
        if img is None:
            continue
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        images.append(img.astype('float32') / 255.0)
        labels.append(label)
    return images, labels

real_images, real_labels = load_images_from_directory(f"{DATASET_PATH}/real", 0)
fake_images, fake_labels = load_images_from_directory(f"{DATASET_PATH}/fake", 1)

X = np.array(real_images + fake_images)
y = np.array(real_labels + fake_labels)



Extracting fake: 100%|███████████████████████████████████████████████████| 50/50 [08:18<00:00,  9.98s/it]


In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# === Обучение модели ===
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=BATCH_SIZE
)




2025-04-24 19:10:44.571426: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 858249600 exceeds 10% of free system memory.
2025-04-24 19:10:49.338917: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 858249600 exceeds 10% of free system memory.


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 164ms/step - accuracy: 0.7433 - loss: 0.5770 - val_accuracy: 0.4800 - val_loss: 19.9293
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.8630 - loss: 0.3626 - val_accuracy: 0.8100 - val_loss: 1.6712
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 0.9075 - loss: 0.2463 - val_accuracy: 0.5800 - val_loss: 1.2931
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.9375 - loss: 0.2077 - val_accuracy: 0.8300 - val_loss: 0.3641
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 0.9613 - loss: 0.1261 - val_accuracy: 0.7400 - val_loss: 0.9598
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.9791 - loss: 0.1002 - val_accuracy: 0.9700 - val_loss: 0.2138
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 119ms/step - accuracy: 0.9812 - loss: 0.0976 - val_accuracy: 0.5300 - val_loss: 5.2013
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.9561 - loss: 

In [11]:
# === Сохранение модели ===
model.save('xception_dfdc_trained.keras')
print("\nМодель обучена и сохранена в файл xception_dfdc_trained.h5")


Модель обучена и сохранена в файл xception_dfdc_trained.h5


In [12]:
# DeepFake Detection using XceptionNet (Keras version)
import cv2
import numpy as np
import face_recognition
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf



# === Настройки ===
IMG_SIZE = 299
MODEL_PATH = 'xception_dfdc_trained.keras'  # Предобученная модель

# === Загрузка обученной модели ===
model = tf.keras.models.load_model(MODEL_PATH)

# === Преобразование для входа ===
def preprocess_face(face):
    face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
    face = face.astype('float32') / 255.0
    return np.expand_dims(face, axis=0)

# === Извлечение лица ===
def extract_face(frame):
    face_locations = face_recognition.face_locations(frame)
    if not face_locations:
        return None
    top, right, bottom, left = face_locations[0]
    face = frame[top:bottom, left:right]
    return face

# === Анализ видео ===
def analyze_video(video_path, max_frames=15):
    cap = cv2.VideoCapture(video_path)
    predictions = []
    frame_count = 0

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        face = extract_face(frame)
        if face is None:
            continue
        input_face = preprocess_face(face)
        pred = model.predict(input_face, verbose=0)[0][0]
        predictions.append(pred)
        frame_count += 1
    cap.release()
    if predictions:
        return np.mean(predictions)
    else:
        return None

# === Пример использования ===
video_real = 'data/RealVideo.mp4'  # Замените на путь к своему видео
video_fake = 'data/FakeReal.mp4'  # Замените на путь к своему видео

real_score = analyze_video(video_real)
fake_score = analyze_video(video_fake)

print(f"\nРеальное видео — вероятность фейка: {real_score:.2f}")
print(f"Фейковое видео — вероятность фейка: {fake_score:.2f}")



Реальное видео — вероятность фейка: 0.34
Фейковое видео — вероятность фейка: 1.00
